In [ ]:
import pandas as pd
import json

filename = 'yelp_academic_dataset_review.json'

# https://stackoverflow.com/questions/73567315/using-dask-to-parallelize-read-json-save-parquet
reader = pd.read_json(filename, orient='records', lines=True, chunksize=100000)
i=1
for chunk in reader:
    chunk.to_parquet(f'part{i:02d}.parquet')
    i = i+1
# This step takes about 1.5 hours for chunksize =6990000

In [2]:
# https://stackoverflow.com/questions/33813815/how-to-read-a-parquet-file-into-pandas-dataframe
df1 = pd.read_parquet('part01.parquet', engine='pyarrow')
df1.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [3]:
df2 = pd.read_parquet('part02.parquet', engine='pyarrow')
df2.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
6990000,nN1EWUFB6G5vXLCGMnFpow,V5jfkywq-f4FpvDrUJMdfw,Zg83olSjsdXAhE5EEy5IcQ,5,1,0,0,This has been one of our favorite restaurants ...,2014-07-19 05:25:16
6990001,gs6qP-N4oV-Fr8dpqowiIQ,l3YDgKIUhVLXKq8fYOR5NA,YGXh3EWuaZHWucYfyxQFUg,5,25,2,0,I recently graduated from Galen and passed my ...,2016-10-23 15:59:02
6990002,e0BM_gXdhJzswPSWEizItA,Kho08pWNfeGnRbZTnMKy3w,KoE-tbMBF33yHhuALmIezw,4,4,1,4,Spavia Day Spa was just the pampering that I n...,2019-04-09 15:52:12
6990003,n3JX4UuW_TY59a3wIG0tiQ,ooHF-vRzlkyN9UEVMKocAQ,5rSg5FARufPsy_ACLLhy-g,4,14,4,11,Chewing over which bakery in Tampa to visit fo...,2021-12-26 12:32:24
6990004,6RvdhphGmuAkBX8Yf4kYKw,VIx1vpiXv3xAZ3sE8qGk2Q,9sOHuGGwb8uRVu7pANSWfg,4,1,0,1,Brought a group of 23 high school students to ...,2019-02-20 14:20:20


In [4]:
df1.shape

(6990000, 9)

In [5]:
df1.dropna(inplace=True)
df1.shape

(6990000, 9)

In [6]:
df2.shape

(280, 9)

In [7]:
df2.dropna(inplace=True)
df2.shape

(280, 9)

In [8]:
# https://stackoverflow.com/questions/69462253/how-to-combine-two-datasets-vertically-in-pandas
# Concatenate two dfs, it loses column names
data = [df1, df2]

df_master = pd.concat([dfi.rename({old: new for new, old in enumerate(dfi.columns)}, axis=1) for dfi in data], ignore_index=True)


df_master

# Drop rows to prepare to import into SQL business table

# Then use SQL to query 93101 and write into new Santa Barbara Table

,0,1,2,3,4,5,6,7,8
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


In [9]:
# Rename columns
df_master = df_master.rename(columns={0: 'review_id', 1: 'user_id', 2: 'business_id', 3: 'stars', 4: 'useful', 5: 'funny', 6: 'cool', 7: 'text', 8: 'date'})
df_master.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [10]:
df_master.drop(['user_id', 'useful', 'funny', 'cool', 'date'], inplace=True, axis=1)
df_master.head(3)

,review_id,business_id,stars,text
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...


In [17]:
df_master= df_master.replace(r'\n','\\n', regex=True)

In [21]:
df_master = df_master.loc[df_master['review_id'] != '4mjHdYusDWIWIYowXILnHA']

In [22]:
df_master = df_master.loc[df_master['review_id'] != 'VA2dpQ1iq9Ujw8ctUpDlgw']

In [24]:
df_master = df_master.loc[df_master['review_id'] != 'PLDAkiRIFIbVqovEKBus2w']

In [27]:
df_master = df_master.loc[df_master['review_id'] != 'zllQbkpiS1-f9DTV4H_eug']

In [28]:
df_master.shape

(6990276, 4)

In [29]:
df_master.to_csv(r'Final_Reviews.csv', index=False)